In [ ]:
###Поиск признаков, которые могут обобщаться на многоязычные данные (минимум 10 признаков рассмотрено,, обучать можно любую модель, оценивать нужно на validation.csv)  - 3 балла

In [ ]:
Некоторые идеи позаимстованы у Сергея Косяка

In [102]:
import pandas as pd
import re
import sklearn

In [103]:
train1 = pd.read_csv('toxic-comment-train.csv')
validation = pd.read_csv('validation_with_translation.csv')

In [ ]:
Отфильтруем в валидации только те фрагменты, где перевод выполнен, чтобы сопоставление было корректным (некоторые тексты не перевелись)

In [ ]:
Признаки, которые нужно считать на оригинале, даже если используется перевод (либо предположительно не зависят от языка)

In [104]:
def has_link_or_email(text): #на основе кода Сергея Косяка, но он не подсчитывал емейлы
    if len(re.findall("https?://[^ ]+", text)) + len(re.findall('[^ ]@', text)) > 0: #после непробела, чтобы не считать тэги пользователей соцсети
        return True
    else:
        return False

In [105]:
def punctuation_blocs(text): #количество блоков пунктуации, несколько знаков препинания подряд считаются за один
    return len(re.findall('[\.,\?!—]+', text))

In [106]:
def expressive_punctuation(text): #есть ли несколько вопросительных/восклицательных знаков подряд
    return len(re.findall('[\?!]{2,}', text))

In [107]:
punctuation = '!"(),-.:;?—«»'

In [108]:
def longest_same_nonpunctuation_char(text): #на основе кода Сергея Косяка, но он и пунктуацию подсчитывал
    length = 0
    curr_length = 1
    for i in range(1,len(text)):
        if text[i] == text[i-1]:
            if text[i] not in punctuation:
                curr_length += 1
            else:
                if curr_length > length:
                    length = curr_length
                    curr_length = 1
    return length

In [109]:
def digit_sequences(text):
    return len(re.findall('[0-9]+', text))

In [110]:
def sentenize(text): #код Сергея Косяка
    sents = re.split("[\n.!?]( |$)", text)
    return [s for s in sents if len(s) > 0 and s != " "]

In [111]:
def caps_ratio_not_sent_begining(text): #на основе кода Сергея Косяка, 
    sents = sentenize(text) #но я не учитываю заглавные буквы в начале предложений
    non_begin_sym = len(text) - len(sents)
    caps = 0
    for sent in sents:
        if len(sent) > 1:
            for letter in sent[1:]:
                if letter not in punctuation and letter != " ":
                    if letter.lower() != letter:
                        caps += 1
    if non_begin_sym:
        return caps/non_begin_sym
    else:
        return 0

In [112]:
def splitted_into_paragraphs(text):
    paragraphs = text.split('\n')
    if len(paragraphs) > 1:
        return True
    else:
        return False

Признаки, которые могут различаться в оригинале и в английском переводе

In [113]:
#Для подсчёта длины предложения в словах и средней длины слова нужна токенизация
def tokenize(text): #код Сергея Косяка
    text = re.sub(f"[{punctuation}]", " ", text)
    text = re.sub("\s+", " ", text)
    return text.split()

In [114]:
def mean_sent_len(text):
    return len(tokenize(text))/len(sentenize(text))

In [115]:
def duplicates(tokens): #доля повторяющихся слов
    if tokens:
        return (len(tokens) - len(set(tokens)))/len(tokens)
    else:
        return 0

In [116]:
from statistics import mean
def mean_word_len(tokens):
    if tokens:
        return mean([len(token) for token in tokens])
    else:
        return 0

In [ ]:
Оформим тренировочный датасет

In [117]:
train1['tokens'] = train1['comment_text'].apply(tokenize)

train1['has_link_or_em'] = train1['comment_text'].apply(has_link_or_email)
train1['punctuation_blocs'] = train1['comment_text'].apply(punctuation_blocs)
train1['expressive_punctuation'] = train1['comment_text'].apply(expressive_punctuation)
train1['longest_same_nonpunctuation_char'] = train1['comment_text'].apply(longest_same_nonpunctuation_char)
train1['digit_sequences'] = train1['comment_text'].apply(digit_sequences)
train1['capitalization'] = train1['comment_text'].apply(caps_ratio_not_sent_begining)
train1['splitted_into_paragraphs'] = train1['comment_text'].apply(splitted_into_paragraphs)
train1['mean_sent_len'] = train1['comment_text'].apply(mean_sent_len)
train1['duplicates'] = train1['tokens'].apply(duplicates)
train1['mean_word_len'] = train1['tokens'].apply(mean_word_len)

Обучим модель. Цель - посмотреть, как соотносится результат на английском и на иноязычеых данных, а не достичь высокого качества, так что параметры по умолчанию подойдут

In [118]:
X = train1[['has_link_or_em', 'punctuation_blocs', 'expressive_punctuation', 'longest_same_nonpunctuation_char', 'digit_sequences', 'capitalization', 'splitted_into_paragraphs', 'mean_sent_len', 'duplicates', 'mean_word_len']]
y = train1['toxic']

In [119]:
from sklearn.linear_model import LogisticRegression

In [120]:
model = LogisticRegression()

In [121]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y)

In [122]:
model.fit(X_train, y_train)

C:\Users\lizan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\lizan\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [123]:
prediction = model.predict(X_test)
prediction_proba = model.predict_proba(X_test)[:,1]

In [124]:
sklearn.metrics.f1_score(y_test, prediction)

0.0085949177877429

In [125]:
sklearn.metrics.roc_auc_score(y_test, prediction)

0.502053706958425

In [126]:
from sklearn.tree import DecisionTreeClassifier

In [127]:
model2 = DecisionTreeClassifier()

In [128]:
model2.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [129]:
prediction2 = model2.predict(X_test)
prediction2_proba = model2.predict_proba(X_test)[:,1]

In [130]:
sklearn.metrics.f1_score(y_test, prediction2)

0.21183046110852355

In [131]:
sklearn.metrics.roc_auc_score(y_test, prediction2_proba)

0.5925938081723932

Вообще-то это работает ужасно, но посмотрим, станет ли ещё хуже на многоязычных данных

Попробуем везде использовать исходный текст

In [132]:
really_translated['tokens'] = really_translated['comment_text'].apply(tokenize)

really_translated['has_link_or_em'] = really_translated['comment_text'].apply(has_link_or_email)
really_translated['punctuation_blocs'] = really_translated['comment_text'].apply(punctuation_blocs)
really_translated['expressive_punctuation'] = really_translated['comment_text'].apply(expressive_punctuation)
really_translated['longest_same_nonpunctuation_char'] = really_translated['comment_text'].apply(longest_same_nonpunctuation_char)
really_translated['digit_sequences'] = really_translated['comment_text'].apply(digit_sequences)
really_translated['capitalization'] = really_translated['comment_text'].apply(caps_ratio_not_sent_begining)
really_translated['splitted_into_paragraphs'] = really_translated['comment_text'].apply(splitted_into_paragraphs)
really_translated['mean_sent_len'] = really_translated['comment_text'].apply(mean_sent_len)
really_translated['duplicates'] = really_translated['tokens'].apply(duplicates)
really_translated['mean_word_len'] = really_translated['tokens'].apply(mean_word_len)

C:\Users\lizan\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\lizan\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\lizan\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

In [133]:
X_valid_original = really_translated[['has_link_or_em', 'punctuation_blocs', 'expressive_punctuation', 'longest_same_nonpunctuation_char', 'digit_sequences', 'capitalization', 'splitted_into_paragraphs', 'mean_sent_len', 'duplicates', 'mean_word_len']]
y_valid = really_translated['toxic']

In [135]:
prediction_original1 = model.predict(X_valid_original)
prediction_original1_proba = model.predict_proba(X_valid_original)[:,1]

In [136]:
sklearn.metrics.f1_score(y_valid, prediction_original1)

0.0048250904704463205

In [137]:
sklearn.metrics.roc_auc_score(y_valid, prediction_original1_proba)

0.6704683028835571

In [139]:
prediction_original2_proba = model2.predict_proba(X_valid_original)[:,1]
prediction_original2 = model2.predict(X_valid_original)

In [140]:
sklearn.metrics.f1_score(y_valid, prediction_original2)

0.20502092050209209

In [141]:
sklearn.metrics.roc_auc_score(y_valid, prediction_original2_proba)

0.5361404908650671

In [ ]:
На иноязычном оригинале rocout не хуже, чем на английских текстах

Используем перевод там, где предполагается различие

In [142]:
really_translated['tokens'] = really_translated['translation'].apply(tokenize)

really_translated['mean_sent_len'] = really_translated['translation'].apply(mean_sent_len)
really_translated['duplicates'] = really_translated['tokens'].apply(duplicates)
really_translated['mean_word_len'] = really_translated['tokens'].apply(mean_word_len)

C:\Users\lizan\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\lizan\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\lizan\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

In [143]:
X_valid_translated = really_translated[['has_link_or_em', 'punctuation_blocs', 'expressive_punctuation', 'longest_same_nonpunctuation_char', 'digit_sequences', 'capitalization', 'splitted_into_paragraphs', 'mean_sent_len', 'duplicates', 'mean_word_len']]

In [144]:
prediction_translated1 = model.predict(X_valid_translated)
prediction_translated1_proba = model.predict_proba(X_valid_translated)[:,1]

In [145]:
sklearn.metrics.f1_score(y_valid, prediction_translated1)

0.007228915662650603

In [146]:
sklearn.metrics.roc_auc_score(y_valid, prediction_translated1_proba)

0.6355317246313008

In [147]:
prediction_translated2 = model2.predict(X_valid_translated)
prediction_translated2_proba = model2.predict_proba(X_valid_translated)[:,1]

In [148]:
sklearn.metrics.f1_score(y_valid, prediction_translated2)

0.20625415834996674

In [149]:
sklearn.metrics.roc_auc_score(y_valid, prediction_translated2_proba)

0.5341376843495488

На переводе работает немного хуже (но не сильно).
rocauc выше при использовании логистической регрессии, f-мера - при использовании дерева решений